In [12]:

from dateutil import parser
import json
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
import requests
import json
from string import Template
import pandas as pd




In [ ]:
symbols = ['IBM']

url = Template('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=$symbol&interval=5min&apikey=5LZM13OZQMZFPU92')

for symbol in symbols:
    r = requests.get(url.substitute(symbol=symbol))
    data = r.json()

    print(data)



In [20]:
#make json to dataframe

df = pd.DataFrame(data['Time Series (5min)']).T
df

,1. open,2. high,3. low,4. close,5. volume
2023-01-19 19:50:00,140.7200,140.7200,140.7200,140.7200,151
2023-01-19 18:50:00,140.6200,140.6200,140.6200,140.6200,610
2023-01-19 18:45:00,140.6500,140.6500,140.6000,140.6400,744
2023-01-19 17:45:00,140.7999,140.7999,140.7999,140.7999,254
2023-01-19 17:35:00,140.8000,140.8000,140.6200,140.6200,524
...,...,...,...,...,...
2023-01-19 07:25:00,140.0500,140.0500,140.0200,140.0200,941
2023-01-19 07:10:00,140.1400,140.1400,140.1400,140.1400,501
2023-01-19 06:40:00,140.1400,140.1600,140.1400,140.1600,1364
2023-01-18 18:40:00,140.2000,140.2000,140.2000,140.2000,100


In [10]:
bucket = "DSI_test"
token ="yxk8_Or5qHZJxrhJE3SkAnTQSViQCsmrUoR0xPZd_0scy1T8FTuL1cKSTDKh1ft8Bqs3Zbt7Rwkys-FzajIVFQ=="
org="DSI"
client = InfluxDBClient(url="http://localhost:8086", token=token, org=org)


point = Point("weather")
with open("weather.json", "r") as json_file:
    data = json.load(json_file)

    point.tag("station", data["name"])
    point.time(parser.parse(data["updated"]))
    for key, value in data["sensors"].items():
        point.field(key, value)



In [11]:

with client.write_api(write_options=SYNCHRONOUS) as writer:
    try:
        writer.write(bucket=bucket, record=[point])
    except InfluxDBError as e:
        print(e)